# Python WinRM

## Install Python WinRM
pip install pywinrm

## Creating a PowerShell Remoting Session with a local administrator

In [ ]:
import winrm
import getpass

computer = input("Enter the computer to connect")
user     = input("Enter your user name")
password = getpass.getpass(prompt='Enter the password for '+str(user), stream=None)

try:
    session  = winrm.Session(computer, auth=('{}\{}'.format(computer,user), password), transport='ntlm')
    result = session.run_ps("$env:computername")
    print("Status Code: " + str(result.status_code))
    print(result.std_out)

except Exception as e:
        print(e)

## Creating a PowerShell Remoting Session with a domain user

In [ ]:
import winrm
import getpass

computer = input("Enter the computer to connect")
user     = input("Enter the userprincipalname (user@domain.net)")
password = getpass.getpass(prompt='Enter the password for '+str(user), stream=None)

try:
    session  = winrm.Session(computer, auth=(user, password), transport='kerberos')
    result = session.run_ps("$env:computername")
    print("Status Code: " + str(result.status_code))
    print(result.std_out)

except Exception as e:
    print(e)

## Run a PowerShell-Script in a PowerShell Remoting Session

In [ ]:
import winrm
import getpass
import json

def invoke_remotescript(computer,user,password,ps_script):
    try:
        session = winrm.Session(computer, auth=('{}\{}'.format(computer,user), password), transport='ntlm')
        result  = session.run_ps(ps_script)
        if result.status_code == 0:
            return { "StatusCode": result.status_code, "Output": result.std_out.decode().replace('\r\n','') }
        else:
            return { "StatusCode": result.status_code, "Error": result.std_err }

    except Exception as e:
        print(e)

ps_script = """
function Get-PyInstalledMemory{
    [CmdletBinding()]
    param(
        [Parameter(Mandatory=$false)]
        [String] $Name
    )
    $ComputerSystem = Get-WmiObject Win32_ComputerSystem
    $TotalMemoryMB  = [Math]::Round($ComputerSystem.TotalPhysicalMemory/1mb)
    return "Total Memory $($TotalMemoryMB) MB"
}
Get-PyInstalledMemory
"""

computer  = input("Enter the computer to connect")
user      = input("Enter your user name")
password  = getpass.getpass(prompt='Enter the password for '+str(user), stream=None)

scriptret = invoke_remotescript(computer,user,password,ps_script)
json_object = json.dumps(scriptret, indent = 4)   
print(json_object)